In [1]:
# Understanding autoencoders via a simple example

## generate the same dataset

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l


/home/sobhan/miniconda3/envs/ml-dl/lib/python3.9/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
# I feel that there should be a relationship between columns in dataset.  not just random. Let's see if I'm right?
## Comment: after figuring this in my mind, I realized it is always true to have a realtionship between features
## in simple words, yeah they must have a relationship...!

In [3]:
first_col_random = torch.rand(10,1) # producing a random column
features = torch.cat((first_col_random, 2*first_col_random), 1) # that is how a relationship between cols 
# are introduced here in my example.

In [4]:
# now, I want to have more columns. each colmn is multiplied from first column
first_col_random = torch.rand(1000,1) 
features = first_col_random # initial values for the fist col
for i in range(2,5):
    features = torch.cat((features, i*first_col_random), 1) # creation of other colmn

In [5]:
# see? they are multipied from the first col
features

tensor([[0.8835, 1.7670, 2.6504, 3.5339],
        [0.5175, 1.0349, 1.5524, 2.0698],
        [0.1425, 0.2851, 0.4276, 0.5702],
        ...,
        [0.9415, 1.8829, 2.8244, 3.7658],
        [0.9987, 1.9974, 2.9961, 3.9948],
        [0.2838, 0.5676, 0.8514, 1.1352]])

In [6]:
# first_col_random

In [7]:
# in auto encoder we want a reperesentation of our data. The best representation here is the one
# shows that colmns are multiplied from the first one (abvious). But this one seems like to be 
# an unsupervised learning. But no. Let's think differently. We want a representation x_ of our data x
# so, I guess now it is a supervised learning. We only need to define ground truth as y = x and
# come up with wieghts and biases (through training) to best fit this.

# based on the explanation above, let's figure this out:
labels = features

In [8]:
features.shape, labels.shape

(torch.Size([1000, 4]), torch.Size([1000, 4]))

In [9]:
features[0], labels[0]

(tensor([0.8835, 1.7670, 2.6504, 3.5339]),
 tensor([0.8835, 1.7670, 2.6504, 3.5339]))

## Reading the data set

In [10]:
def load_array(data_arrays, batch_size, is_train=True): #@save
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


In [11]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [12]:
data_iter

In [13]:
# it is import to use iterator for that
next(iter(data_iter))

[tensor([[0.2952, 0.5904, 0.8856, 1.1808],
         [0.8962, 1.7925, 2.6887, 3.5849],
         [0.1158, 0.2316, 0.3474, 0.4633],
         [0.2953, 0.5906, 0.8858, 1.1811],
         [0.8624, 1.7248, 2.5871, 3.4495],
         [0.4134, 0.8268, 1.2402, 1.6536],
         [0.9247, 1.8494, 2.7742, 3.6989],
         [0.3546, 0.7092, 1.0638, 1.4183],
         [0.2100, 0.4199, 0.6299, 0.8398],
         [0.3952, 0.7904, 1.1856, 1.5808]]),
 tensor([[0.2952, 0.5904, 0.8856, 1.1808],
         [0.8962, 1.7925, 2.6887, 3.5849],
         [0.1158, 0.2316, 0.3474, 0.4633],
         [0.2953, 0.5906, 0.8858, 1.1811],
         [0.8624, 1.7248, 2.5871, 3.4495],
         [0.4134, 0.8268, 1.2402, 1.6536],
         [0.9247, 1.8494, 2.7742, 3.6989],
         [0.3546, 0.7092, 1.0638, 1.4183],
         [0.2100, 0.4199, 0.6299, 0.8398],
         [0.3952, 0.7904, 1.1856, 1.5808]])]

## Defining the Model

In [14]:
# The bottleneck here is only of dim 2

In [15]:
import torch.nn as nn
import torch as T

class Net(T.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hid1 = T.nn.Linear(4, 2)  # 6-(10-10)-3
        self.oupt = T.nn.Linear(2, 4)
    

        T.nn.init.xavier_uniform_(self.hid1.weight)
        T.nn.init.zeros_(self.hid1.bias)
        T.nn.init.xavier_uniform_(self.oupt.weight)
        T.nn.init.zeros_(self.oupt.bias)

    def forward(self, x):
        z = T.tanh(self.hid1(x))
        z = self.oupt(z)  # no softmax: CrossEntropyLoss() 
        
        return z

In [16]:
net = Net()

In [17]:
# model:
net

Net(
  (hid1): Linear(in_features=4, out_features=2, bias=True)
  (oupt): Linear(in_features=2, out_features=4, bias=True)
)

## Initializing Model Parameters

In [18]:
# net[0] has accesss to the first layer of net

# net[0].weight.data.normal_(0, 0.01)
# net[0].bias.data.fill_(0)

## Defining the Loss Function

In [19]:
loss = nn.MSELoss()

In [20]:
# testing loss function:
X = torch.tensor([[-1.2373,  0.3100, -1.4755, -1.1790]])
y = X
# net(X)
loss(net(X), y)

tensor(1.5142, grad_fn=<MseLossBackward0>)

## Defining the Optimization Algorithm


In [21]:
# Minibatch stochastic gradient descent is a standard tool for optimizing neural networks and thus
# PyTorch supports it alongside a number of variations on this algorithm in the optim module. When
# we instantiate an SGD instance, we will specify the parameters to optimize over (obtainable from
# our net via net.parameters()), with a dictionary of hyperparameters required by our optimization
# algorithm. Minibatch stochastic gradient descent just requires that we set the value lr, which is
# set to 0.03 here.

trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [22]:
net.parameters()

<generator object Module.parameters at 0x7f5591528c80>

In [23]:
list(net.parameters())

[Parameter containing:
 tensor([[ 0.2037,  0.3724, -0.4606,  0.8351],
         [ 0.4188,  0.3827, -0.6066,  0.2799]], requires_grad=True),
 Parameter containing:
 tensor([0., 0.], requires_grad=True),
 Parameter containing:
 tensor([[ 0.5064, -0.4418],
         [ 0.9196, -0.7782],
         [-0.8896, -0.2414],
         [ 0.1819, -0.0038]], requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0., 0.], requires_grad=True)]

In [24]:
num_epochs = 200
for epoch in range(num_epochs):
    for X,Y in data_iter:
#         print(X,Y)
        l = loss(net(X), Y)
        trainer.zero_grad()
        l.backward()
        trainer.step() #optimizer
    l = loss(net(features), labels)
#     print(net[0].weight.data)
    if (epoch%10 == 0):
        print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.185904
epoch 11, loss 0.007006
epoch 21, loss 0.003356
epoch 31, loss 0.001762
epoch 41, loss 0.001240
epoch 51, loss 0.001158
epoch 61, loss 0.001083
epoch 71, loss 0.001411
epoch 81, loss 0.001136
epoch 91, loss 0.000847
epoch 101, loss 0.000693
epoch 111, loss 0.000642
epoch 121, loss 0.000652
epoch 131, loss 0.000611
epoch 141, loss 0.000653
epoch 151, loss 0.000547
epoch 161, loss 0.001121
epoch 171, loss 0.000507
epoch 181, loss 0.000512
epoch 191, loss 0.000494


In [25]:
# how our model represent a data sample like this: [0.1, 0.2, 0.3, 0.4]
x = torch.tensor([0.1, 0.2, 0.3, 0.4])
net(x)

tensor([0.0996, 0.1989, 0.3279, 0.4004], grad_fn=<AddBackward0>)

In [26]:
# That was just a simple example with linear relationship between colmns. 
# How about more complex relationships?